In [1]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip '/content/drive/MyDrive/Colab_Notebooks/data/for_task.zip'

Archive:  /content/drive/MyDrive/Colab_Notebooks/data/for_task.zip
   creating: for_task/
  inflating: for_task/Barracuda.tsv  
  inflating: for_task/Boss.tsv       
  inflating: for_task/Dance_Till_Dead.tsv  
  inflating: for_task/FEV_Reject.tsv  
  inflating: for_task/Johny.tsv      
  inflating: for_task/King_Charles.tsv  
  inflating: for_task/Lucky_Number_5.tsv  
  inflating: for_task/Lychelle.tsv   
  inflating: for_task/Matchstick_Man.tsv  
  inflating: for_task/Mengsk.tsv     
  inflating: for_task/Pavlina_Grey.tsv  
  inflating: for_task/Starry_Sky.tsv  
  inflating: for_task/The_Wide_Pirate.tsv  
  inflating: for_task/vdjdb.slim.txt  
  inflating: for_task/Wing_And_A_Prayer.tsv  


# Прочтение таблиц

Для последующей кластеризации необходимо сократить размер данных пациента. Для этого мы сортируем таблицу по убыванию числа ридов и оставляем первые 10000 значений.

In [6]:
df = pd.read_csv("/content/for_task/Dance_Till_Dead.tsv", sep='\t')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,4669,0.002789,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,4046,0.002417,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
2,2,2965,0.001771,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
3,3,2628,0.001570,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,2078,0.001241,TGTGCCAGCAGCCAAGGGACAGCCTACGAGCAGTACTTC,CASSQGTAYEQYF,TRBV4-2,TRBJ2-7


In [7]:
database = pd.read_excel("/content/drive/MyDrive/Colab_Notebooks/data/vdjdb_slim.xlsx")

In [8]:
database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [9]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,4669,0.002789,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,4046,0.002417,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
2,2,2965,0.001771,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
3,3,2628,0.001570,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,2078,0.001241,TGTGCCAGCAGCCAAGGGACAGCCTACGAGCAGTACTTC,CASSQGTAYEQYF,TRBV4-2,TRBJ2-7
...,...,...,...,...,...,...,...
9995,8982,9,0.000005,TGTGCCAGCAGTTTAACGGCTAGCGGCAACCACAATGAGCAGTTCTTC,CASSLTASGNHNEQFF,TRBV12-4,TRBJ2-1
9996,9495,9,0.000005,TGTGCCAGCAGTTTAAGACAGGGGAACACTGAAGCTTTCTTT,CASSLRQGNTEAFF,TRBV12-4,TRBJ1-1
9997,9768,9,0.000005,TGTGCCAGCAGTTTAAGCGGGGGAGAGACCCAGTACTTC,CASSLSGGETQYF,TRBV12-3,TRBJ2-5
9998,9494,9,0.000005,TGTGCCAGCAGTTTAAGCGGGGGGGTAAATGAGCAGTTCTTC,CASSLSGGVNEQFF,TRBV12-3,TRBJ2-1


# Функция кластеризации

Мы предполагаем, что в таблице с антигенами (database) могут остатья конкретные последовательности аминокислот, но в экспериментальных данных могут быть CDR3 отличные на 1 аминокислоту. Такие виды клонов скорее всего могут так же реагировать на патоген, но при этом упускаться при сравнении с database. Поэтому мы проводим кластеризацию.

In [10]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [11]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df


,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,4669,0.002789,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1,1
1,1,4046,0.002417,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1,2
2,2,2965,0.001771,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5,3
3,3,2628,0.001570,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4,4
4,4,2078,0.001241,TGTGCCAGCAGCCAAGGGACAGCCTACGAGCAGTACTTC,CASSQGTAYEQYF,TRBV4-2,TRBJ2-7,5
...,...,...,...,...,...,...,...,...
9995,8982,9,0.000005,TGTGCCAGCAGTTTAACGGCTAGCGGCAACCACAATGAGCAGTTCTTC,CASSLTASGNHNEQFF,TRBV12-4,TRBJ2-1,8525
9996,9495,9,0.000005,TGTGCCAGCAGTTTAAGACAGGGGAACACTGAAGCTTTCTTT,CASSLRQGNTEAFF,TRBV12-4,TRBJ1-1,1966
9997,9768,9,0.000005,TGTGCCAGCAGTTTAAGCGGGGGAGAGACCCAGTACTTC,CASSLSGGETQYF,TRBV12-3,TRBJ2-5,58
9998,9494,9,0.000005,TGTGCCAGCAGTTTAAGCGGGGGGGTAAATGAGCAGTTCTTC,CASSLSGGVNEQFF,TRBV12-3,TRBJ2-1,8360


Каждую последовательность из сокращённых данных мы отнесли к определённому кластеру. Теперь необходимо определиться с выбором наиболее значимых последовательностей.

# Подсчёт Total Read Proportion

In [12]:

# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})



# Сортировка по Sequence Count

Количество схожих последовательностей в каждом кластере важно, но необходимо учитывать частоту.

In [13]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
57,58,0.002178,178
4,5,0.002449,89
97,98,0.001237,65
48,49,0.000654,34
73,74,0.000582,32
...,...,...,...
3064,3072,0.000013,1
3061,3069,0.000013,1
3060,3068,0.000013,1
3059,3067,0.000013,1


# Сортировка по Total Read Proportion

In [14]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
4,5,0.002449,89
57,58,0.002178,178
97,98,0.001237,65
7,8,0.000897,3
6,7,0.000888,2
...,...,...,...
8329,8336,0.000011,2
8353,8360,0.000011,2
8276,8283,0.000011,2
8312,8319,0.000011,2


# Выбор кластера

In [15]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [16]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,20,CASSLVLADNEQFF,0.000526,TRBV7-8,TRBJ2-1,TRBV27*01,TRBJ2-1*01,MHCI,HomoSapiens,CMV,pp65
1,55,CASSSGQAYEQYF,0.000294,TRBV6-2,TRBJ2-7,"TRBV7-8*01,TRBV11-1*01","TRBJ2-7*01,TRBJ2-1*01",MHCI,HomoSapiens,EBV,EBNA3A
2,71,CASSLEGQASSYEQYF,0.000240,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
3,71,CASSLEGQASSYEQYF,0.000240,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
4,83,CASSLGGGDNEQFF,0.000220,TRBV27,TRBJ2-1,TRBV7-3*01,TRBJ2-1*01,MHCI,HomoSapiens,InfluenzaA,M
...,...,...,...,...,...,...,...,...,...,...,...
681,5,CASSYGYEQYF,0.000005,TRBV12-3,TRBJ2-7,TRBV19*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,EBNA4
682,8500,CASSSRTGGYGYTF,0.000005,TRBV12-3,TRBJ1-2,TRBV19*01,TRBJ1-2*01,MHCI,HomoSapiens,HIV-1,Gag
683,50,CASSSRDRGYEQYF,0.000005,TRBV27,TRBJ2-7,TRBV7-9*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1
684,5,CASSWGSYEQYF,0.000005,TRBV6-2,TRBJ2-7,TRBV28*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1


In [17]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
2,71,CASSLEGQASSYEQYF,0.000240,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
3,71,CASSLEGQASSYEQYF,0.000240,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
21,71,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
22,71,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
28,229,CASSDGGATDTQYF,0.000105,TRBV6-4,TRBJ2-3,TRBV6-4*01,TRBJ2-3*01,MHCI,HomoSapiens,HomoSapiens,MLANA
...,...,...,...,...,...,...,...,...,...,...,...
601,5,CASSLDSYEQYF,0.000005,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
609,8142,CASSQDRGGGYTF,0.000005,TRBV3-1,TRBJ1-2,TRBV3-1*01,TRBJ1-2*01,MHCI,HomoSapiens,HIV-1,Gag
614,50,CASSRDRGYEQYF,0.000005,TRBV25-1,TRBJ2-7,TRBV25-1*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
623,58,CASSASYEQYF,0.000005,TRBV7-2,TRBJ2-7,TRBV7-2*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1


In [19]:
full_vj['Cluster'].value_counts()

,count
Cluster,
58,10
71,6
1824,6
7328,4
119,3
583,3
229,3
7412,3
98,3


# Обработка для одного выбранного кластера

In [20]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 58]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
57,57,465,0.000278,TGTGCCAGCAGCTTTTTCACAGATACGCAGTATTTT,CASSFFTDTQYF,TRBV7-2,TRBJ2-3,58
699,703,76,0.000045,TGTGCCAGCAGCTTATTTACAGATACGCAGTATTTT,CASSLFTDTQYF,TRBV7-2,TRBJ2-3,58
841,843,65,0.000039,TGTGCCAGCAGCTTATTCGCAGATACGCAGTATTTT,CASSLFADTQYF,TRBV7-2,TRBJ2-3,58
1340,1356,44,0.000026,TGTGCCAGCAGCCCCTTCACAGATACGCAGTATTTT,CASSPFTDTQYF,TRBV7-2,TRBJ2-3,58
1342,1355,44,0.000026,TGTGCCAGCAGCCTTAGCACAGATACGCAGTATTTT,CASSLSTDTQYF,TRBV7-9,TRBJ2-3,58
...,...,...,...,...,...,...,...,...
9834,10007,9,0.000005,TGTGCCAGCAGTCCGAACACAGATACGCAGTATTTT,CASSPNTDTQYF,TRBV12-3,TRBJ2-3,58
9861,10205,9,0.000005,TGTGCCAGCAGTGAAGAGACCCAGTACTTC,CASSEETQYF,TRBV2,TRBJ2-5,58
9944,10143,9,0.000005,TGTGCCAGCAGTTACTTGGAGACCCAGTACTTC,CASSYLETQYF,TRBV6-5,TRBJ2-5,58
9945,10142,9,0.000005,TGTGCCAGCAGTTATAAAGAGACCCAGTACTTC,CASSYKETQYF,TRBV12-3,TRBJ2-5,58


In [21]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [22]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,58,CASSLFTDTQYF,0.000045,TRBV7-2,TRBJ2-3,TRBV7-8*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,BZLF1
1,58,CASSLSTDTQYF,0.000026,TRBV7-9,TRBJ2-3,TRBV12-3*01,TRBJ2-3*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
2,58,CASSLSTDTQYF,0.000026,TRBV7-9,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
3,58,CASSLSTDTQYF,0.000026,TRBV7-9,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
4,58,CASSLSTDTQYF,0.000026,TRBV7-9,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA3A
...,...,...,...,...,...,...,...,...,...,...,...
123,58,CASSLGTDTQYF,0.000005,TRBV7-2,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
124,58,CASSLGTDTQYF,0.000005,TRBV7-2,TRBJ2-3,TRBV12-3*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
125,58,CASSLGGEQYF,0.000005,TRBV7-2,TRBJ2-7,TRBV27*01,TRBJ2-7*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
126,58,CASSLGGEQYF,0.000005,TRBV7-2,TRBJ2-7,TRBV29*01,TRBJ2-7*01,MHCI,MusMusculus,InfluenzaA,PA


# Разборки с V и J

In [23]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
24,58,CASSLSTDTQYF,0.000013,TRBV5-4,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
25,58,CASSLSTDTQYF,0.000013,TRBV5-4,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA3A
46,58,CASSQETQYF,0.000010,TRBV3-1,TRBJ2-5,TRBV3-1*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,IE1
59,58,CASSLGETQYF,0.000010,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
62,58,CASSLGETQYF,0.000010,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
79,58,CASSLGETQYF,0.000008,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
82,58,CASSLGETQYF,0.000008,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
86,58,CASSLGTDTQYF,0.000008,TRBV7-9,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
102,58,CASSLGTDTQYF,0.000007,TRBV12-3,TRBJ2-3,TRBV12-3*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
111,58,CASSASYEQYF,0.000005,TRBV7-2,TRBJ2-7,TRBV7-2*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,IE1


In [26]:
len(full_vj[full_vj['antigen.species'] == 'CMV'])/ len(full_vj)

0.7

In [32]:
# confidence
0.002178	 / proportions_df_sorted['Total Read Proportion'].sum() * len(full_vj[full_vj['antigen.species'] == 'CMV'])/ len(full_vj)

0.033360861110314964